<a href="https://colab.research.google.com/github/Blane187/Project/blob/main/RVC_CLI_infer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center> <h1> RVC_CLI<br>

<center> inference only <br>


Created by [Blaise](https://github.com/blaise-tk) with [Vidal](https://github.com/Vidalnt) and [Poopmaster](https://github.com/poiqazwsx). Based on [RVC_CLI](https://github.com/blaise-tk/RVC_CLI).

- Colab inspired on [RVC v2 Disconnected](https://colab.research.google.com/drive/1XIPCP9ken63S7M6b5ui1b36Cs17sP-NS).

### If you restart the runtime, run it again.

In [ ]:
%cd /content/RVC_CLI

# Installation

In [ ]:
# @title Mount Google Drive
from google.colab import drive

drive.mount("/content/drive")

In [ ]:
# @title install
!git clone https://github.com/blaise-tk/RVC_CLI --quiet
%cd /content/RVC_CLI

!pip install -r requirements.txt --quiet

!python main.py prerequisites

# Infer

In [ ]:
 #@title Download Model

import os
import zipfile
import shutil
import urllib.request

BASE_DIR = os.getcwd()
rvc_models_dir = os.path.join(BASE_DIR, 'logs')

def extract_zip(extraction_folder, zip_name):
    os.makedirs(extraction_folder)
    with zipfile.ZipFile(zip_name, 'r') as zip_ref:
        zip_ref.extractall(extraction_folder)
    os.remove(zip_name)

    index_filepath, model_filepath = None, None
    for root, dirs, files in os.walk(extraction_folder):
        for name in files:
            if name.endswith('.index') and os.stat(os.path.join(root, name)).st_size > 1024 * 100:
                index_filepath = os.path.join(root, name)

            if name.endswith('.pth') and os.stat(os.path.join(root, name)).st_size > 1024 * 1024 * 40:
                model_filepath = os.path.join(root, name)

    if not model_filepath:
        raise Exception(f'No .pth model file was found in the extracted zip. Please check {extraction_folder}.')

    # move model and index file to extraction folder
    os.rename(model_filepath, os.path.join(extraction_folder, os.path.basename(model_filepath)))
    if index_filepath:
        os.rename(index_filepath, os.path.join(extraction_folder, os.path.basename(index_filepath)))

    # remove any unnecessary nested folders
    for filepath in os.listdir(extraction_folder):
        if os.path.isdir(os.path.join(extraction_folder, filepath)):
            shutil.rmtree(os.path.join(extraction_folder, filepath))

def download_online_model(url, dir_name):
    try:
        print(f'[~] Downloading voice model with name {dir_name}...')
        zip_name = url.split('/')[-1]
        extraction_folder = os.path.join(rvc_models_dir, dir_name)
        if os.path.exists(extraction_folder):
            raise Exception(f'Voice model directory {dir_name} already exists! Choose a different name for your voice model.')

        if 'pixeldrain.com' in url:
            url = f'https://pixeldrain.com/api/file/{zip_name}'

        urllib.request.urlretrieve(url, zip_name)

        print('[~] Extracting zip...')
        extract_zip(extraction_folder, zip_name)
        print(f'[+] {dir_name} Model successfully downloaded!')

    except Exception as e:
        raise Exception(str(e))

url = "https://huggingface.co/HRVC/Blaze/resolve/main/Blaze-The-Cat.zip?download=true" # @param {type:"string"}
dir_name = "Blaze" # @param {type:"string"}

download_online_model(url, dir_name)

In [ ]:
# @title Run Inference
# @markdown Please upload the audio file to your Google Drive path `/content/drive/MyDrive` and specify its name here. For the model name, use the zip file name without the extension. Alternatively, you can check the path `/content/RVC_CLI/logs` for the model name (name of the folder).

import os

current_dir = os.getcwd()

model_name = "Blaze"  # @param {type:"string"}
model_folder = os.path.join(current_dir, f"logs/{model_name}")

if not os.path.exists(model_folder):
    raise FileNotFoundError(f"Model directory not found: {model_folder}")

files_in_folder = os.listdir(model_folder)
pth_path = next((f for f in files_in_folder if f.endswith(".pth")), None)
index_file = next((f for f in files_in_folder if f.endswith(".index")), None)

if pth_path is None or index_file is None:
    raise FileNotFoundError("No model found.")

pth_file = os.path.join(model_folder, pth_path)
index_file = os.path.join(model_folder, index_file)

input_path = "/content/drive/MyDrive/Vocales/baka_mitai.wav"  # @param {type:"string"}
output_path = "/content/baka_mitai_output.wav" # @param {type:"string"}
export_format = "WAV"  # @param ['WAV', 'MP3', 'FLAC', 'OGG', 'M4A'] {allow-input: false}
f0method = "hybrid[rmvpe+fcpe]"  # @param ["pm", "dio", "crepe", "crepe-tiny", "harvest", "rmvpe", "fcpe", "hybrid[rmvpe+fcpe]"] {allow-input: false}
f0up_key = 13  # @param {type:"slider", min:-24, max:24, step:0}
filter_radius = 3  # @param {type:"slider", min:0, max:10, step:0}
rms_mix_rate = 0.8  # @param {type:"slider", min:0.0, max:1.0, step:0.1}
protect = 0.5  # @param {type:"slider", min:0.0, max:0.5, step:0.1}
index_rate = 0.7  # @param {type:"slider", min:0.0, max:1.0, step:0.1}
hop_length = 128  # @param {type:"slider", min:1, max:512, step:0}
clean_strength = 0.3  # @param {type:"slider", min:0.0, max:1.0, step:0.1}
split_audio = False  # @param{type:"boolean"}
clean_audio = False  # @param{type:"boolean"}
autotune = True  # @param{type:"boolean"}

!python main.py infer --f0up_key "{f0up_key}" --filter_radius "{filter_radius}" --index_rate "{index_rate}" --hop_length "{hop_length}" --rms_mix_rate "{rms_mix_rate}" --protect "{protect}" --f0autotune "{autotune}" --f0method "{f0method}" --input_path "{input_path}" --output_path "{output_path}" --pth_path "{pth_file}" --index_path "{index_file}" --split_audio "{split_audio}" --clean_audio "{clean_audio}" --clean_strength "{clean_strength}" --export_format "{export_format}"

from IPython.display import Audio, display, clear_output

clear_output()
display(Audio(output_path, autoplay=True))